In [1]:
import os

In [2]:
%pwd

'd:\\ML_project_with_MLFlow\\Complete-ML-Project-with-ML-Flow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML_project_with_MLFlow\\Complete-ML-Project-with-ML-Flow'

In [5]:
#
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tejaswini1999teju15/Complete-ML-Project-with-ML-Flow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="tejaswini1999teju15"
os.environ["MLFLOW_TRACKING_PASSWORD"]="cc67f8b5467bbf2bf69794d3f1e8d40572f3b506"

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)

class ModelEvaluationConfig:
    root_dir : Path
    test_data_path : Path
    model_path : Path
    all_params : dict
    metric_file_name : Path
    target_column : str
    mlflow_uri : str

In [13]:
from src.mlproject.constants import *
from box import ConfigBox
from src.mlproject.utils.common import read_yaml, create_directories

class ConfigurationManager4:
    def __init__(
            self,
            # config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(Path("config/config.yaml"))
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params= self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        
        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/tejaswini1999teju15/Complete-ML-Project-with-ML-Flow.mlflow"
            

        )
        return model_evaluation_config



In [14]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import json

In [15]:
from src.mlproject.utils.common import save_json
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self,actual,pred):
        RMSE=np.sqrt(mean_squared_error(actual,pred))
        MAE = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)

        return RMSE,MAE,r2
    
    def log_into_mlflow(self):
        

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    

    

                 


In [16]:
try:
    config=ConfigurationManager4()
    model_eval_config=config.get_model_evaluation_config()
    model_eval = ModelEvaluation(config=model_eval_config)
    model_eval.log_into_mlflow()

except Exception as e:
    raise e
    


[2024-03-12 19:34:51,189: INFO: common :yaml_file : config\config.yaml loaded successfully]
[2024-03-12 19:34:51,193: INFO: common :yaml_file : params.yaml loaded successfully]
[2024-03-12 19:34:51,201: INFO: common :yaml_file : schema.yaml loaded successfully]
[2024-03-12 19:34:51,203: INFO: common :created directory at artifacts]
[2024-03-12 19:34:51,206: INFO: common :created directory at artifacts/model_evaluation]


NameError: name 'save_json' is not defined